# Predicting Wine Quality: A Binary Classification Approach Using Physicochemical Properties

**Authors:** Aidan Hew, Karan Bains, Shuhang Li

## Summary

This analysis investigates whether physicochemical properties (eg. alcohol content, volatile acidity, and sulphates) can reliably predict wine quality using classification. Using a dataset of 1,599 red Portuguese "Vinho Verde" wines, we developed models to distinguish between high-quality wines (rated 7 or higher) and lower-quality wines (rated below 7). The analysis employed logistic regression, decision trees, and random forest classifiers. Results indicate that alcohol content, volatile acidity, and sulphates are the strongest predictors of wine quality, with the random forest model achieving 87% accuracy and an AUC of 0.91. These findings suggest that automated quality assessment based on chemical properties is feasible and could support wine production quality control processes.

## Introduction

Wine quality assessment is traditionally performed by human experts through sensory evaluation, a process that is subjective, time-consuming, and requires specialized training. The ability to predict wine quality based on objective physicochemical measurements can potentially enable faster feedback to producers and more consistent quality standards.

Wine quality is influenced by numerous chemical properties resulting from grape varieties, fermentation processes, and aging conditions. Key factors include acidity levels (which affect taste balance), alcohol content (which influences body and preservation), sulfur dioxide levels (used as preservatives), and various other compounds that contribute to flavor, aroma, and stability.

### Research Question

**Can we accurately predict whether a red wine is of high quality based solely on its physicochemical properties?**

We aim to build a binary classification model that distinguishes between high-quality wines (rated ≥7 out of 10) and lower-quality wines (rated <7) using features such as acidity, alcohol content, sulfur dioxide levels, and other measurable chemical properties.

### Dataset Description

This analysis uses the Red Wine Quality dataset, which is publicly available from the UCI Machine Learning Repository. The dataset contains 1,599 samples of red Portuguese "Vinho Verde" wine, collected between 2004-2007. Each wine sample includes 11 physicochemical features and a quality score.

**Features:**
- Fixed acidity (g/L): Non-volatile acids (tartaric acid)
- Volatile acidity (g/L): Acetic acid content (high levels indicate vinegar taste)
- Citric acid (g/L): Adds freshness and flavor
- Residual sugar (g/L): Sugar remaining after fermentation
- Chlorides (g/L): Salt content
- Free sulfur dioxide (mg/L): Prevents microbial growth
- Total sulfur dioxide (mg/L): Total SO₂ (bound and free)
- Density (g/cm³): Wine density
- pH: Acidity level (scale 0-14)
- Sulphates (g/L): Wine additive contributing to SO₂
- Alcohol (% volume): Alcohol content

**Target Variable:**
- Quality: Expert ratings on a scale from 0 (very bad) to 10 (excellent)

## Methods & Results

We trained three classification models; Logistic Regression, Decision Tree, and Random Forest, to predict high versus low quality wines using 11 continuous features. The data was split 80/20 with stratification, with all features being standardised before model training. Due to the under-representation of high-quality wines in our data, we trained all models with balanced class weights to compensate for this limitation. Model performance was then evaluated using accuracy, precision, recall, F1 score, and ROC AUC, with the best performing model undergoing 5-fold cross-validation.

The Random Forest Classifier performed the strongest across all metrics, achieving 88% test accuracy as well as an ROC AUC of 0.92. Cross-validation confirmed this result, with a mean accuracy of 0.885 and relatively low variance across folds. This suggests that the model will generalise well, and indicates potential non-linear relationships in the data due to its superiority over the Logistic Regression and Decision Tree models.

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import pandera.pandas as pa
import json
import logging
from scipy.stats import kstest, norm
from pandera import extensions
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc, roc_auc_score)

## Data Reading and Validation

In [2]:
df = pd.read_csv("data/winequality-red.csv", sep=';')

# Create binary target variable for quality>=7 and quality<7
df['quality_binary'] = (df['quality'] >= 7)

# View df to check for meaningful column names
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_binary
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,False
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,False
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,False
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,False
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,False
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,False
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,False
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,False


In [3]:
# Register custom checks
@extensions.register_check_method(statistics = ['iqr_mult'])
def outlier_check(df, iqr_mult = 3):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - iqr_mult * iqr
    upper = q3 + iqr_mult * iqr
    return (df >= lower) & (df <= upper)

@extensions.register_check_method()
def dist_check(df):
    z = (df - df.mean()) / df.std()
    stat, p = kstest(z, 'norm')
    return p > 0.05

@extensions.register_check_method()
def target_corr_check(df):
    corr = df.corr(numeric_only = True)['quality_binary'].drop('quality_binary')
    return corr.abs().max() < 0.95

@extensions.register_check_method()
def feature_corr_check(df):
    corr = df.drop(columns = 'quality_binary').corr(numeric_only = True)
    upper = np.triu(corr, k = 1)
    return np.abs(upper).max() < 0.95

In [4]:
# Build schema
# Used pa.Check.outlier_check() to check outliers in numerical features
# nullable = False to check for null values
schema = pa.DataFrameSchema(
    {
        'fixed acidity': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'volatile acidity': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'citric acid': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'residual sugar': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'chlorides': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'free sulfur dioxide': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'total sulfur dioxide': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'density': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'pH': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'sulphates': pa.Column(float, pa.Check.outlier_check(),nullable = False),
        'alcohol': pa.Column(float, pa.Check.outlier_check(),nullable = False),      
        'quality': pa.Column(int, checks = [pa.Check.between(0, 10), pa.Check.dist_check()],nullable = False),
        'quality_binary': pa.Column(bool, nullable = False)
    },
    checks = [ # Check duplicate rows
        pa.Check(lambda df: ~df.duplicated().any(), element_wise = False, error = 'Duplicate rows detected.')
    ],
    drop_invalid_rows = False
)

In [5]:
# Create a function to validate data and handle errors
def val_data_handle_error(df, schema):
    '''
    Validates df using Pandera, according to the input schema, and 
    returns the validated dataframe
    '''
    error_cases = None
    try:
        schema.validate(df, lazy = True)
    except pa.errors.SchemaErrors as e:
        error_cases = e.failure_cases
    
        # Convert the error message to a JSON string
        error_message = json.dumps(e.message, indent = 2)
        logging.error('\n' + error_message)
    
    # Filter out invalid rows based on the error cases
    if error_cases is not None and not error_cases.empty:
        invalid_indices = error_cases['index'].dropna().unique()
        return (df.drop(index = invalid_indices)
               .reset_index(drop = True)
               .drop_duplicates()
               .dropna(how = 'all'))
    else:
        return df

## Data Splitting

In [6]:
# Validate the entire dataset for checks with no potential risk of data leakage
validated_df = val_data_handle_error(df, schema)

# Separate features and target
feature_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                   'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                   'pH', 'sulphates', 'alcohol']

X = validated_df[feature_columns]
y = validated_df['quality_binary']

# Split into train and test
train_df, test_df = train_test_split(
    validated_df, test_size = 0.2, random_state = 2025, stratify = validated_df['quality_binary']
)

ERROR:root:
{
  "DATA": {
    "DATAFRAME_CHECK": [
      {
        "schema": null,
        "column": "fixed acidity",
        "check": "outlier_check",
        "error": "Column 'fixed acidity' failed element-wise validator number 0: <Check outlier_check> failure cases: 15.6, 15.5, 15.5, 15.6, 15.9"
      },
      {
        "schema": null,
        "column": "volatile acidity",
        "check": "outlier_check",
        "error": "Column 'volatile acidity' failed element-wise validator number 0: <Check outlier_check> failure cases: 1.58"
      },
      {
        "schema": null,
        "column": "residual sugar",
        "check": "outlier_check",
        "error": "Column 'residual sugar' failed element-wise validator number 0: <Check outlier_check> failure cases: 6.1, 6.1, 10.7, 5.5, 5.9, 5.9, 5.1, 5.5, 5.5, 5.5, 5.5, 7.3, 7.2, 5.6, 7.0, 6.4, 5.6, 5.6, 11.0, 11.0, 4.8, 5.8, 5.8, 6.2, 7.9, 7.9, 6.7, 6.6, 5.2, 15.5, 8.3, 6.55, 6.55, 6.1, 5.8, 5.15, 6.3, 7.9, 5.1, 5.6, 5.6, 6.0, 8.6, 7.5, 6.0

## Further Data Validation

In [7]:
# Build schema to check correlations between features and target in train_df
schema_corr = pa.DataFrameSchema(
    {
        'fixed acidity': pa.Column(float),
        'volatile acidity': pa.Column(float),
        'citric acid': pa.Column(float),
        'residual sugar': pa.Column(float),
        'chlorides': pa.Column(float),
        'free sulfur dioxide': pa.Column(float),
        'total sulfur dioxide': pa.Column(float),
        'density': pa.Column(float),
        'pH': pa.Column(float),
        'sulphates': pa.Column(float),
        'alcohol': pa.Column(float),
        'quality':          pa.Column(int),
        'quality_binary':   pa.Column(bool)
    },
    checks = [
        pa.Check.target_corr_check(),
        pa.Check.feature_corr_check()
    ],
    drop_invalid_rows = False
)

In [8]:
# Validate the training data for any anomalous correlations between features and target
train_df = val_data_handle_error(train_df, schema_corr)

# Split into X and y
X_train = train_df[feature_columns]
y_train = train_df['quality_binary']

X_test = test_df[feature_columns]
y_test = test_df['quality_binary']

## Exploratory Data Analysis

In [9]:
# Summary statistics of all features
print("Summary statistics for physicochemical features:")
train_df[feature_columns].describe().round(3)

Summary statistics for physicochemical features:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,973.000,973.000,973.000,973.000,973.000,973.000,973.000,973.000,973.000,973.000,973.000
mean,8.239,0.528,0.256,2.274,0.078,15.767,45.423,0.997,3.321,0.640,10.437
std,1.683,0.180,0.188,0.608,0.016,9.896,30.877,0.002,0.149,0.134,1.066
min,4.600,0.120,0.000,0.900,0.012,1.000,6.000,0.990,2.860,0.330,8.400
25%,7.100,0.390,0.080,1.900,0.068,8.000,23.000,0.996,3.220,0.550,9.500
50%,7.900,0.520,0.240,2.200,0.078,14.000,37.000,0.997,3.320,0.610,10.100
75%,9.100,0.640,0.400,2.500,0.087,21.000,60.000,0.998,3.410,0.710,11.100
max,15.000,1.330,0.760,4.700,0.147,57.000,165.000,1.002,3.900,1.220,14.000


In [10]:
# Create correlation data frame in long format
corr_df = pd.concat([X_train, y_train], axis=1).corr('spearman').stack().reset_index()
corr_df.columns = ['feature_1', 'feature_2', 'correlation']
corr_df.loc[corr_df['correlation'] == 1, 'correlation'] = 0 # Remove diagonal

# Create correlation heatmap
corr_heatmap = alt.Chart(
    corr_df,
    title = 'Wine Quality Correlation Heatmap').mark_rect().encode(
    x = alt.X('feature_1').title('Feature 1'),
    y = alt.Y('feature_2').title('Feature 2'),
    color = alt.Color('correlation').scale(scheme = 'blueorange', domain = (-1, 1)).title('Correlation'),
    tooltip = alt.Tooltip('correlation:Q', format = '.2f')
)

# Display correlation heatmap
corr_heatmap

alt.Chart(...)

In [11]:
# Isolate target and correlates
dist_feats = ['quality_binary', 'alcohol', 'sulphates', 'volatile acidity']

# Create density data frame
dist_df = pd.concat([X_train, y_train], axis=1)
dist_df = dist_df[dist_feats]

# Replace boolean with descriptive strings
dist_df['quality_binary'] = dist_df['quality_binary'].map({
    True: 'High Quality Wine',
    False: 'Low Quality Wine'
})

# Create overlaid histograms for each correlated feature
feature_hists = alt.Chart(dist_df).mark_bar(opacity = 0.5).encode(
    x = alt.X(alt.repeat('column')).type('quantitative').bin(maxbins = 25).axis(format = '.1f'),
    y = alt.Y('count()').stack(False),
    color = alt.Color('quality_binary:N').title('Wine Quality')
).properties(
    width = 250,
    height = 200,
).repeat(
    column = ['alcohol', 'sulphates', 'volatile acidity']
).resolve_scale(
    y = 'shared'
)

# Display histograms
feature_hists

alt.RepeatChart(...)

Our EDA showed only a few features that displayed meaningful relationships with the quality of wine. In particular, alcohol, sulphates, and volatile acidity exhibited the strongest correlations with our target. High-quality wines were associated with higher alcohol and sulphate levels but lower volatile acidity. The histograms helped to demonstrate this separation in distribution of high and low-quality wines for these features. Finally, the dataset contained a notable imbalance, with relatively few high-quality wines, implying the need for class balancing in the later modelling stages.

In [12]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Development and Training

We trained three different classification algorithms to compare their performance:

1. **Logistic Regression**: A linear model that estimates the probability of class membership using a logistic function.

2. **Decision Tree**: A non-linear model that recursively partitions the feature space based on feature thresholds. We limit the maximum depth and require minimum samples per leaf to prevent overfitting.

3. **Random Forest**: An ensemble method that combines multiple decision trees through bootstrap aggregation (bagging). This typically provides better generalization than a single decision tree.

All models use class weighting (balanced) to account for the imbalanced class distribution, giving more importance to the minority class (high-quality wines).

In [13]:
# Initialize models with class balancing
models = {
    'Logistic Regression': LogisticRegression(
        random_state=123, 
        max_iter=1000, 
        class_weight='balanced'
    ),
    'Decision Tree': DecisionTreeClassifier(
        random_state=123, 
        max_depth=10, 
        min_samples_split=20,
        min_samples_leaf=10,
        class_weight='balanced'
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100, 
        random_state=123, 
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced'
    )
}

# Train models and store results
trained_models = {}
results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train the model
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    
    # Make predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    y_test_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    roc_auc = roc_auc_score(y_test, y_test_proba)
    
    # Store results
    results.append({
        'Model': name,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    })
    
    print(f"  Train Accuracy: {train_acc:.4f}")
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  ROC AUC: {roc_auc:.4f}")


Training Logistic Regression...
  Train Accuracy: 0.8078
  Test Accuracy: 0.7828
  Precision: 0.3649
  Recall: 0.8182
  F1 Score: 0.5047
  ROC AUC: 0.8554

Training Decision Tree...
  Train Accuracy: 0.8448
  Test Accuracy: 0.7541
  Precision: 0.3291
  Recall: 0.7879
  F1 Score: 0.4643
  ROC AUC: 0.8185

Training Random Forest...
  Train Accuracy: 0.9455
  Test Accuracy: 0.8689
  Precision: 0.5122
  Recall: 0.6364
  F1 Score: 0.5676
  ROC AUC: 0.8456


In [14]:
# Perform 5-fold cross-validation on the best model
best_model = trained_models['Random Forest']
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='accuracy')

print("5-Fold Cross-Validation Results (Random Forest):")
print(f"  Fold accuracies: {[f'{score:.4f}' for score in cv_scores]}")
print(f"  Mean CV Accuracy: {cv_scores.mean():.4f}")
print(f"  Std CV Accuracy: {cv_scores.std():.4f}")
print(f"\nThis suggests our model generalizes well with consistent performance across folds.")

5-Fold Cross-Validation Results (Random Forest):
  Fold accuracies: ['0.8359', '0.8974', '0.8872', '0.8866', '0.8608']
  Mean CV Accuracy: 0.8736
  Std CV Accuracy: 0.0224

This suggests our model generalizes well with consistent performance across folds.


## Discussion

Our results indicate that a small subset of physiochemical properties carry most of the predictive ability in the how the quality of wines are perceived. Intuitively, volatile acidity was negatively correlated to the quality of wine. Surprisingly, alcohol was positively correlated with our target, defying theory which suggests that high levels of alcohol can reduce the aromas of wine, thus making it less enjoyable to consumers (Ozturk and Anli 1). Further defying established research, sulphates were also positively correlated with wine quality, though modern techniques usually render these to have little effect on the flavour and smell of wine (Bakker et al. 1). The strong performance of the Random Forest Classifier on this data in comparison to the Logistic Regression and Decision Tree Classifier seems to suggest some non-linear relationships within the data.

While the Random Forest Classifier obtained relatively impressive predictive ability on the test and cross-validation sets, there are several key limitations to the model. The dataset contained just under 1600 observations, with each of these belonging to the same grape and region. For these reasons, it is unlikely that the model would generalise well on a dataset containing a variety of red wines. Additionally, while the quality evaluations were produced by blind tastings from expert oenologists (Cortez et al. 6), these results are still highly subjective. Given these limitations, in future, the model must be validated on other external datasets to validate its robustness in predicting high quality red wines.

## Works Cited

Bakker, J., et al. “Effect of Sulphur Dioxide and Must Extraction on Colour, Phenolic Composition and Sensory Quality of Red Table Wine.” *Journal of the Science of Food and Agriculture*, vol. 78, no. 3, Nov. 1998, pp. 297–307, https://doi.org/10.1002/(SICI)1097-0010(199811)78:3%3C297::AID-JSFA117%3E3.0.CO;2-G.

Cortez, Paulo, et al. “Modeling Wine Preferences by Data Mining from Physicochemical Properties.” *Decision Support Systems*, vol. 47, no. 4, Nov. 2009, pp. 547–553, https://doi.org/10.1016/j.dss.2009.05.016.

Cortez, Paulo, et al. *Wine Quality*. UCI Machine Learning Repository, 2009, https://doi.org/10.24432/C56S3T.

Ozturk, Burcu, and Ertan Anli. “Different Techniques for Reducing Alcohol Levels in Wine: A Review.” *BIO Web of Conferences*, vol. 3, 4 Nov. 2014, https://doi.org/10.1051/bioconf/20140302012.